https://www.rankedchoicevoting.org/data_clearinghouse
    
https://drive.google.com/drive/folders/1DJzIrTaDW3GSGJTkPTGAlpAMbozFG_pm
    
Objective for this notebook: separate the elections into the following categories:
* Leading candidate in the first round has greater than 50% first choice votes
* Leading candidate in the first round has between 45-50% first choice votes
* Leading candidate in the first round has less than 45% of first choice votes

In [1]:
import glob
import pandas
print('pandas',pandas.__version__)

pandas 0.23.4


# import data

In [2]:
list_of_files = glob.glob('voting_data/Pierce_County/Pierce County/*')
len(list_of_files)

8

In [3]:
list_of_files

['voting_data/Pierce_County/Pierce County/Pierce County Auditor 2009 Ballot Image.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Executive 2008 Master Lookup.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Assessor - Treasurer 2008 Ballot Image.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Council, District No. 2 2008 Master Lookup.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Council, District No. 2 2008 Ballot Image.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Assessor - Treasurer 2008 Master Lookup.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Executive 2008 Ballot Image Data.txt',
 'voting_data/Pierce_County/Pierce County/Pierce County Auditor 2009 Master Lookup.txt']

In [4]:
list_of_ballot_files=[]
for filename in list_of_files:
    if filename.endswith('.txt'):
        with open(filename,'r') as fil:
            file_contents = fil.readlines()
        if len(file_contents[0].strip())==45:
            print(filename)
            list_of_ballot_files.append(filename)
            print(file_contents[1])

voting_data/Pierce_County/Pierce County/Pierce County Auditor 2009 Ballot Image.txt
000071400001543600000010050000002002000044000

voting_data/Pierce_County/Pierce County/Pierce County Assessor - Treasurer 2008 Ballot Image.txt
000019200006315800000010050000002002000000001

voting_data/Pierce_County/Pierce County/Pierce County Council, District No. 2 2008 Ballot Image.txt
000019300007697700000010050000063002000013100

voting_data/Pierce_County/Pierce County/Pierce County Executive 2008 Ballot Image Data.txt
000019700006315800000010050000002002000000001



In [5]:
reslts={'leading candidate in first round has more than 50% of first choice votes':[],
        'leading candidate in first round vote has between 50% and 45% of first choice votes':[],
        'leading candidate in first round vote has less than 45% of first choice votes':[]}

for ballot in list_of_ballot_files:
    #print(ballot)
    df = pandas.read_fwf(ballot,
                     header=None,
                     widths=[7,9,7,3,7,3,7,1,1])
    df.columns=['contest_id','pref_voter_id',
            'serial_number','tally_type_id',
            'precinct_id','vote_rank',
            'candidate_id','over_vote','under_vote']
    #print(df.shape)
    #print(df['candidate_id'].unique())
    df_cand = df[df['candidate_id']!=0] # drop rows where no candidate is specified
    series_of_candidates_and_first_choice_count = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()
    number_of_first_choice_votes =  series_of_candidates_and_first_choice_count.sum()
    #print('number of first choice votes:',number_of_first_choice_votes)
    if (series_of_candidates_and_first_choice_count > number_of_first_choice_votes*0.5).any():
        #print('Leading candidate in the first round has greater than 50% first choice votes')
        reslts['leading candidate in first round has more than 50% of first choice votes'].append(ballot)
    elif ((series_of_candidates_and_first_choice_count <= number_of_first_choice_votes*0.5).any() and 
          (series_of_candidates_and_first_choice_count >= number_of_first_choice_votes*0.45).any()):
        #print('Leading candidate in the first round has between 45-50% first choice votes')
        reslts['leading candidate in first round vote has between 50% and 45% of first choice votes'].append(ballot)
    elif (series_of_candidates_and_first_choice_count < number_of_first_choice_votes*0.45).any():
        #print('Leading candidate in the first round has less than 45% of first choice votes')
        reslts['leading candidate in first round vote has less than 45% of first choice votes'].append(ballot)
    else:
        raise Exception('should not reach this condition')

In [7]:
for k,v in reslts.items():
    print(k,':')
    for election in v:
        print('   ',election.replace('voting_data/','').replace('.txt',''))

leading candidate in first round has more than 50% of first choice votes :
leading candidate in first round vote has between 50% and 45% of first choice votes :
    Pierce_County/Pierce County/Pierce County Auditor 2009 Ballot Image
    Pierce_County/Pierce County/Pierce County Council, District No. 2 2008 Ballot Image
leading candidate in first round vote has less than 45% of first choice votes :
    Pierce_County/Pierce County/Pierce County Assessor - Treasurer 2008 Ballot Image
    Pierce_County/Pierce County/Pierce County Executive 2008 Ballot Image Data
